<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

*   Understand the dataset for Chicago Public School level performance
*   Store the dataset in an Db2 database on IBM Cloud instance
*   Retrieve metadata about tables and columns and query data from mixed case columns
*   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01&download=true&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**:

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR.

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next".

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database


In [2]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa

  Using cached SQLAlchemy-1.3.9-cp37-cp37m-win_amd64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.23
    Uninstalling SQLAlchemy-1.3.23:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\sqlalchemy-1.3.23.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



In [3]:
%load_ext sql

In [4]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://mgr61801:aG6rnocuLnVjRZCo@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb?security=SSL

'Connected: mgr61801@bludb'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [7]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select TABSCHEMA, TABNAME, CREATE_TIME FROM SYSCAT.TABLES WHERE TABSCHEMA = 'MGR61801'

 * ibm_db_sa://mgr61801:***@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb
Done.


tabschema,tabname,create_time
MGR61801,TRUCKS,2021-09-12 12:45:06.739354
MGR61801,COUNTRY1,2021-09-10 14:17:30.363072
MGR61801,PROVINCES,2021-09-10 13:51:34.375516
MGR61801,TEST,2021-09-10 14:23:20.789515
MGR61801,COUNTRY,2021-09-10 14:23:20.944910
MGR61801,PETSALE,2021-09-10 14:45:31.268509
MGR61801,EMPLOYEES,2021-09-10 15:18:15.696119
MGR61801,JOB_HISTORY,2021-09-10 15:18:15.980906
MGR61801,JOBS,2021-09-10 15:18:16.309437
MGR61801,DEPARTMENTS,2021-09-10 15:18:16.576408


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [35]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select * from sysCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://mgr61801:***@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb
Done.


tabschema,tabname,colname,colno,typeschema,typename,length,scale,typestringunits,stringunitslength,DEFAULT,NULLS,codepage,collationschema,collationname,logged,compact,colcard,high2key,low2key,avgcollen,keyseq,partkeyseq,nquantiles,nmostfreq,numnulls,target_typeschema,target_typename,scope_tabschema,scope_tabname,source_tabschema,source_tabname,dl_features,special_props,hidden,inline_length,pctinlined,IDENTITY,rowchangetimestamp,GENERATED,text,compress,avgdistinctperpage,pagevarianceratio,sub_count,sub_delim_length,avgcollenchar,implicitvalue,seclabelname,rowbegin,rowend,transactionstartid,pctencoded,avgencodedcollen,qualifier,func_path,randdistkey,remarks
MGR61801,SCHOOLS,SCHOOL_ID,0,SYSIBM,INTEGER,4,0,None,None,None,Y,0,None,None,,,-1,    ,    ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,NAME_OF_SCHOOL,1,SYSIBM,VARCHAR,64,0,OCTETS,64,None,Y,1208,SYSIBM,IDENTITY,,,-1,                                 ,                                 ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,"Elementary, Middle, or High School",2,SYSIBM,VARCHAR,2,0,OCTETS,2,None,Y,1208,SYSIBM,IDENTITY,,,-1,  ,  ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,STREET_ADDRESS,3,SYSIBM,VARCHAR,29,0,OCTETS,29,None,Y,1208,SYSIBM,IDENTITY,,,-1,                             ,                             ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,CITY,4,SYSIBM,VARCHAR,7,0,OCTETS,7,None,Y,1208,SYSIBM,IDENTITY,,,-1,       ,       ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,STATE,5,SYSIBM,VARCHAR,2,0,OCTETS,2,None,Y,1208,SYSIBM,IDENTITY,,,-1,  ,  ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,ZIP_CODE,6,SYSIBM,INTEGER,4,0,None,None,None,Y,0,None,None,,,-1,    ,    ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,PHONE_NUMBER,7,SYSIBM,VARCHAR,14,0,OCTETS,14,None,Y,1208,SYSIBM,IDENTITY,,,-1,              ,              ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,LINK,8,SYSIBM,VARCHAR,78,0,OCTETS,78,None,Y,1208,SYSIBM,IDENTITY,,,-1,                                 ,                                 ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
MGR61801,SCHOOLS,NETWORK_MANAGER,9,SYSIBM,VARCHAR,40,0,OCTETS,40,None,Y,1208,SYSIBM,IDENTITY,,,-1,                                 ,                                 ,-1,None,0,-1,-1,-1,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [13]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select distinct(name), coltype, length from sysibm.syscolumns where TBNAME = 'SCHOOLS'

 * ibm_db_sa://mgr61801:***@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb
Done.


name,coltype,length
10th Grade PLAN (2009),VARCHAR,4
10th Grade PLAN (2010),VARCHAR,4
11th Grade Average ACT (2011),VARCHAR,4
9th Grade EXPLORE (2009),VARCHAR,4
9th Grade EXPLORE (2010),VARCHAR,4
ADEQUATE_YEARLY_PROGRESS_MADE_,VARCHAR,3
AVERAGE_STUDENT_ATTENDANCE,VARCHAR,6
AVERAGE_TEACHER_ATTENDANCE,VARCHAR,6
CITY,VARCHAR,7
COLLABORATIVE_NAME,VARCHAR,34


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [ ]:
%sql select*

 * ibm_db_sa://mgr61801:***@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb
Done.


school_id,name_of_school,"Elementary, Middle, or High School",street_address,city,state,zip_code,phone_number,link,network_manager,collaborative_name,adequate_yearly_progress_made_,track_schedule,cps_performance_policy_status,cps_performance_policy_level,healthy_school_certified,safety_icon,safety_score,family_involvement_icon,family_involvement_score,environment_icon,environment_score,instruction_icon,instruction_score,leaders_icon,leaders_score,teachers_icon,teachers_score,parent_engagement_icon,parent_engagement_score,parent_environment_icon,parent_environment_score,average_student_attendance,rate_of_misconducts__per_100_students_,average_teacher_attendance,individualized_education_program_compliance_rate,pk_2_literacy__,pk_2_math__,gr3_5_grade_level_math__,gr3_5_grade_level_read__,gr3_5_keep_pace_read__,gr3_5_keep_pace_math__,gr6_8_grade_level_math__,gr6_8_grade_level_read__,gr6_8_keep_pace_math_,gr6_8_keep_pace_read__,gr_8_explore_math__,gr_8_explore_read__,isat_exceeding_math__,isat_exceeding_reading__,isat_value_add_math,isat_value_add_read,isat_value_add_color_math,isat_value_add_color_read,students_taking__algebra__,students_passing__algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),net_change_explore_and_plan,11th Grade Average ACT (2011),net_change_plan_and_act,college_eligibility__,graduation_rate__,college_enrollment_rate__,college_enrollment,general_services_route,freshman_on_track_rate__,x_coordinate,y_coordinate,latitude,longitude,community_area_number,community_area_name,ward,police_district,location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54,Strong,66,Strong,74,Very Strong,84,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61,NDA,NDA,Average,50,Weak,36,Weak,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.3,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.3,15.5,-0.9,-1.0,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165,1851012.215,41.74711093,-87.73170248,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,56,Average,44,Average,45,Weak,37,Weak,65,Average,48,Average,53,Strong,58,95.50%,10.4,95.80%,100.00%,76.9,NDA,26,24.7,61.8,49.7,39.2,27.2,69.7,60.6,9.1,18.2,11.1,9.6,0.9,2.4,Green,Green,42.9,25,NDA,NDA,NDA,NDA,NDA,NDA,NDA,N

Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where school_type = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [42]:
#% ALTER TABLE SCHOOLS RENAME COLUMN "Elementary, Middle, or High School" TO SCHOOL_Type;

%sql select count(*) from SCHOOLS where school_type = 'ES'

 * ibm_db_sa://mgr61801:***@0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:31198/bludb
Done.


1
462


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score
FROM schools
ORDER BY safety_score
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                        |
| ----------------- | ------- | ----------------- | ----------------------------------------- |
| 2021-07-09        | 2.4     | Malika            | Updated connection string                 |
| 2021-05-19        | 2.3     | Lakshmi Holla     | Updated question                          |
| 2021-04-20        | 2.2     | Malika            | Added the libraries                       |
| 2020-11-27        | 2.1     | Sannareddy Ramesh | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya           | Moved lab to course repo in GitLab        |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
